# 👋 Uji Coba Model Supervised dan Unsupervised

Notebook ini dirancang untuk secara ringkas mendemonstrasikan kemampuan model **prediksi harga jual mobil (Supervised Learning)** dan **segmentasi klaster kendaraan (Unsupervised Learning)** yang sudah dilatih.

Anda akan melihat sebuah contoh input kendaraan dan hasil prediksi harga serta segmen klaster untuk kendaraan tersebut.

## 1. Persiapan: Import Library, Tentukan Jalur, dan Muat Model

Bagian ini akan mengimpor semua library yang dibutuhkan, menentukan lokasi file model dan preprocessor, lalu memuatnya ke memori.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import LabelEncoder # Tambahkan ini
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# --- Tentukan Jalur Model dan Preprocessor ---
# Sesuaikan jalur ini jika file .pkl berada di lokasi yang berbeda.
supervised_model_path = '../models/random_forest_regressor.pkl'
supervised_preprocessor_path = '../models/preprocessor_supervised.pkl'

unsupervised_model_path = '../models/kmeans_clusterer.pkl' 
unsupervised_preprocessor_path = '../models/preprocessor_unsupervised.pkl' 

print("Memuat model dan preprocessor...")
model_supervised = None
preprocessor_supervised = None
supervised_label_encoders = {} 

model_unsupervised = None
preprocessor_unsupervised = None
unsupervised_label_encoders = {} 

try:
    # Muat Supervised Preprocessing Objects
    if os.path.exists(supervised_preprocessor_path):
        loaded_supervised_objects = joblib.load(supervised_preprocessor_path)
        preprocessor_supervised = loaded_supervised_objects['preprocessor']
        supervised_label_encoders = loaded_supervised_objects.get('label_encoders', {}) # Ambil label encoders
        print("✔ Preprocessor supervised dan Label Encoders berhasil dimuat.")
    else:
        print(f"❗ Peringatan: File preprocessor supervised tidak ditemukan di {supervised_preprocessor_path}. Prediksi harga tidak akan berfungsi.")

    if os.path.exists(supervised_model_path):
        model_supervised = joblib.load(supervised_model_path)
        print("✔ Model supervised berhasil dimuat.")
    else:
        print(f"❗ Peringatan: File model supervised tidak ditemukan di {supervised_model_path}. Prediksi harga tidak akan berfungsi.")

    # Muat Unsupervised Preprocessing Objects
    # Asumsi unsupervised preprocessing juga menggunakan format dictionary yang sama
    if os.path.exists(unsupervised_preprocessor_path):
        loaded_unsupervised_objects = joblib.load(unsupervised_preprocessor_path)
        preprocessor_unsupervised = loaded_unsupervised_objects['preprocessor']
        unsupervised_label_encoders = loaded_unsupervised_objects.get('label_encoders', {}) 
        print("✔ Preprocessor unsupervised dan Label Encoders berhasil dimuat.")
    else:
        print(f"❗ Peringatan: File preprocessor unsupervised tidak ditemukan di {unsupervised_preprocessor_path}. Segmentasi klaster tidak akan berfungsi.")

    if os.path.exists(unsupervised_model_path):
        model_unsupervised = joblib.load(unsupervised_model_path)
        print("✔ Model unsupervised berhasil dimuat.")
    else:
        print(f"❗ Peringatan: File model unsupervised tidak ditemukan di {unsupervised_model_path}. Segmentasi klaster tidak akan berfungsi.")

except Exception as e:
    print(f"❌ Terjadi error saat memuat model/preprocessor: {e}")


Memuat model dan preprocessor...
✔ Preprocessor supervised dan Label Encoders berhasil dimuat.
✔ Model supervised berhasil dimuat.
✔ Preprocessor unsupervised dan Label Encoders berhasil dimuat.
✔ Model unsupervised berhasil dimuat.


## 2. Definisikan Fungsi Prediksi dan Segmentasi

Fungsi pembantu ini akan digunakan untuk melakukan prediksi harga dan segmentasi klaster.

In [ ]:
def predict_car_price(input_data_df, preprocessor, model, label_encoders_dict):
    """
    Memprediksi harga jual mobil menggunakan model supervised.
    Input: DataFrame dengan satu baris data baru.
    label_encoders_dict: Dictionary berisi LabelEncoder yang sudah dilatih.
    """
    processed_df = input_data_df.copy()
    try:
        # Terapkan Label Encoding pada kolom 'model', 'trim', 'seller' di input baru
        for col, le in label_encoders_dict.items():
            if col in processed_df.columns:
                try:
                    processed_df[col] = le.transform(processed_df[col].astype(str))
                except ValueError as ve:
                    # print(f"Peringatan LabelEncoder untuk kolom '{col}': {ve}. Nilai tidak dikenal akan di-set ke -1 atau similar.")
                    
                    # Pendekatan fallback: jika kategori tidak dikenal, set ke nilai di luar rentang
                    processed_df[col] = processed_df[col].astype(str).apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
            
        # Transformasi input menggunakan preprocessor_supervised (ColumnTransformer)
        processed_input = preprocessor.transform(processed_df)
        
        # Prediksi harga
        predicted_price = model.predict(processed_input)[0]
        return predicted_price
    except Exception as e:
        print(f"Error saat prediksi harga: {e}")
        return None

def segment_car_cluster(input_data_df, preprocessor, model, label_encoders_dict={}):
    """
    Menentukan segmen klaster mobil menggunakan model unsupervised.
    Input: DataFrame dengan satu baris data baru.
    label_encoders_dict: Dictionary berisi LabelEncoder yang sudah dilatih (opsional, jika unsupervised juga pakai LE).
    """
    processed_df = input_data_df.copy()
    try:
        for col, le in label_encoders_dict.items():
            if col in processed_df.columns:
                try:
                    processed_df[col] = le.transform(processed_df[col].astype(str))
                except ValueError as ve:
                    print(f"Peringatan LabelEncoder untuk kolom unsupervised '{col}': {ve}.")
                    processed_df[col] = processed_df[col].astype(str).apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

        # Transformasi input menggunakan preprocessor_unsupervised
        # Filter kolom yang relevan untuk unsupervised sebelum transform
        # *** Sesuaikan daftar fitur ini gunakan untuk UNsupervised Processing ***
        features_for_unsupervised = [
            'mmr', 'sellingprice', 'year', 'odometer', 'condition', 
            'body', 'make', # Kategorikal yang di-OHE
            'model', 'trim', 'seller' # Jika ini juga digunakan dan sudah di-Label Encode
        ]
        
        # Pastikan hanya fitur yang relevan yang masuk ke preprocessor unsupervised
        # Gunakan .reindex(columns=...) untuk memastikan urutan kolom sama seperti saat training
        input_for_cluster_seg = processed_df[features_for_unsupervised]
        
        processed_input = preprocessor.transform(input_for_cluster_seg)
        
        # Prediksi klaster
        cluster_label = model.predict(processed_input)[0]
        return cluster_label
    except Exception as e:
        print(f"Error saat segmentasi klaster: {e}")
        return None

## 3. Contoh Input Data dan Tampilkan Hasilnya

Di sini, kita akan mendefinisikan satu contoh data mobil baru dan langsung menampilkan hasil prediksi harga serta segmen klaster untuk mobil tersebut.

**❗ PENTING: Sesuaikan nilai input di bawah ini sesuai dengan data mobil yang ingin Anda uji.**
**Pastikan semua kolom yang diperlukan oleh preprocessor supervised dan unsupervised Anda sudah terisi dan memiliki tipe data yang benar.**


In [ ]:
# --- Input Data untuk Prediksi dan Segmentasi ---
# Isi nilai-nilai ini sesuai dengan mobil yang ingin uji.
# Pastikan semua kolom yang digunakan oleh preprocessor ada di sini!

sample_car_data = pd.DataFrame({
    'year': [2019],
    'make': ['Volvo'],
    'model': ['S60'],
    'trim': ['LX'],
    'body': ['Sedan'],
    'transmission': ['automatic'],
    'state': ['CA'],
    'condition': [4.0], 
    'odometer': [67282], 
    'color': ['white'],
    'interior': ['black'],
    'seller': ['volvo na rep/world omni'],
    'mmr': [37500.0],
    'sellingprice': [47750.0] # Tetap sediakan ini jika digunakan sebagai fitur clustering
})

print("### Data Mobil yang Akan Diuji ###")
print(sample_car_data.to_string(index=False))

print("\n" + "="*40)
print("### Hasil Prediksi dan Segmentasi ###")

# --- Prediksi Harga ---
if model_supervised is not None and preprocessor_supervised is not None and supervised_label_encoders:
    # Untuk prediksi harga, 'sellingprice' adalah target, jadi kita bisa menghilangkannya dari input
    input_for_price_pred = sample_car_data.drop(columns=['sellingprice'], errors='ignore')

    # Panggil fungsi prediksi dengan label_encoders
    predicted_price = predict_car_price(input_for_price_pred, preprocessor_supervised, model_supervised, supervised_label_encoders)

    if predicted_price is not None:
        print(f"💰 Prediksi Harga Jual Mobil: ${predicted_price:,.2f}")
    else:
        print("❌ Prediksi harga gagal.")
else:
    print("❗ Model supervised atau preprocessor/label encoders tidak tersedia. Tidak dapat melakukan prediksi harga.")

# --- Segmentasi Klaster ---
cluster_descriptions = {
    0: "Klaster 0 (Lower Tier): Segmen ini didominasi oleh mobil dengan [Harga Jual Terendah], [Kondisi Kurang Baik], dan [Odometer Paling Tinggi]. Ini adalah segmen dengan nilai terendah di pasar bekas.",
    1: "Klaster 1: Segmen ini cenderung memiliki mobil dengan [Harga Jual Sedang], [Kondisi Rata-Rata], dan [Odometer Sedang]. Mereka mewakili sebagian besar pasar bekas dengan nilai yang seimbang.",
    2: "Klaster 2: Klaster ini berisi mobil dengan [Harga Jual Cukup Tinggi], [Kondisi Baik], dan [Odometer Rendah]. Mobil di segmen ini menawarkan keseimbangan antara kualitas dan harga yang lebih premium.",
    3: "Klaster 3 (Upper Tier): Ini adalah segmen mobil premium dengan [Harga Jual Paling Tinggi], [Kondisi Sangat Baik atau Baru], dan [Odometer Paling Rendah]. Mobil di klaster ini mewakili penawaran terbaik atau termahal di pasar."
}
# Asumsi unsupervised juga menggunakan Label Encoders yang sama atau berbeda untuk kolom tertentu
if model_unsupervised is not None and preprocessor_unsupervised is not None:
    cluster_label = segment_car_cluster(sample_car_data, preprocessor_unsupervised, model_unsupervised, unsupervised_label_encoders) # Lewatkan label encoders juga
    if cluster_label is not None:
        print(f"🏷️ Mobil ini masuk ke Klaster: {cluster_label}")

        # Ambil deskripsi dari kamus
        description = cluster_descriptions.get(cluster_label, "Tidak ada deskripsi spesifik untuk klaster ini. Anda bisa menganalisis karakteristik rata-rata dari klaster ini (misalnya dari notebook EDA Anda) untuk memahami segmen pasar mobil ini.")
        print(f"\n👉 {description}")
    else:
        print("❌ Segmentasi klaster gagal.")
else:
    print("❗ Model unsupervised atau preprocessor/label encoders tidak tersedia. Tidak dapat melakukan segmentasi klaster.")

print("\n" + "="*40)

### Data Mobil yang Akan Diuji ###
 year  make model trim  body transmission state  condition  odometer color interior                  seller     mmr  sellingprice
 2019 Volvo   S60   LX Sedan    automatic    CA        4.0     67282 white    black volvo na rep/world omni 37500.0       47750.0

### Hasil Prediksi dan Segmentasi ###
💰 Prediksi Harga Jual Mobil: $36,895.22
🏷️ Mobil ini masuk ke Klaster: 3

👉 Klaster 3 (Upper Tier): Ini adalah segmen mobil premium dengan [Harga Jual Paling Tinggi], [Kondisi Sangat Baik atau Baru], dan [Odometer Paling Rendah]. Mobil di klaster ini mewakili penawaran terbaik atau termahal di pasar.



---